In [3]:
import tensorflow_hub as hub
import tensorflow as tf

def embed_elmo(word_to_embed):
    elmo = hub.Module("https://tfhub.dev/google/elmo/2")
    embedding_tensor = elmo(word_to_embed, as_dict=True)["elmo"]

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        embedding = sess.run(embedding_tensor)
        return sess.run(tf.reduce_mean(embedding,1))

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
import pickle

def load_term(id_file):
    pickle_in = open("Term/"+ str(id_file) +".pickle", "rb")
    return pickle.load(pickle_in)

def save(name, embedding):
    pickle_out = open("elmo_category/elmo-ori/" + name + ".pickle","wb")
    pickle.dump(embedding, pickle_out)
    pickle_out.close()

In [5]:
def run(name):
    models = []
    terms = load_term(cat)
    for term in terms:
        print(term)
        embedding = embed_elmo([term])
        models.append(embedding[0])
    save(name, models)

In [18]:
# category = [
#     'ambience',
#     'food',
#     'service',
#     'price'
# ]
# for cat in category:
#     run(cat)

In [19]:
import pandas as pd

def get_terms(filename):
    df = pd.read_csv('Results/Categorization/' + filename + '.csv')
    for sentence_terms in df['term']:
        if type(sentence_terms) == float:
            continue
        terms = sentence_terms.split('|')
        for term in terms:
            if term == '':
#                 print(filename)
                continue
            all_terms.append(term)

In [20]:
ate = ['lda', 'lda2vec', 'plsa', 'dp']
sim = ['glove', 'w2v', 'fasttext', 'elmo']

all_terms = []
for a in ate:
    for s in sim:
        get_terms(a + '-' + s)
        get_terms(a + '-partial-' + s)
        get_terms(a + '-partial-ner-' + s)

In [21]:
res = sorted(list(dict.fromkeys(all_terms)))

In [22]:
len(res)

621

In [63]:
pickle_out = open("all_terms.pickle","wb")
pickle.dump(res, pickle_out)
pickle_out.close()

In [16]:
pickle_in = open("elmo_restaurant/all_terms/terms/advanatage.pickle", "rb")
print(len(pickle.load(pickle_in)))

1024


In [25]:
def generate_file(part, total):
    with open('run_elmo_ori_'+ str(part) +'.bat',"w", encoding='utf-8') as file:
        file.write('call \"C:\\Users\\Admin\\Anaconda3\\Scripts\\activate.bat\"\ncd \"C:\\Users\\Admin\\tugas-akhir\"\n')
        start = int(len(res) * part / total)
        end = start + 5
        maximum = int(len(res) * (part + 1) / total)
        while True:
            if start > maximum:
                break
            file.write('python elmo_terms.py ' + str(start) + ' ' + str(end) + "\n")
            start = end
            end = start + 5

In [26]:
total = 8
for index in range(0, total):
    generate_file(index, total)